# Llama-Index Hybrid Retriever + Reranking

Hybrid Retrievers are a great way to combine the strenghts of different retrievers. Combined with filtering and reranking, this can be especially powerful in retrieving only the most relevant context from multiple methods. TruLens can take us even farther to highlight the strengths of each component retriever along with measuring the success of the hybrid retriever. This example walks through that process.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truera/trulens/blob/main/trulens_eval/examples/expositional/llama_index/llama_index_hybrid_retriever.ipynb)

## Setup

In [ ]:
# ! pip install trulens_eval==0.24.0 llama_index==0.10.11 openai pypdf torch sentence-transformers

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk-..."

In [ ]:
# Imports main tools:
from trulens_eval import TruLlama, Feedback, Huggingface, Tru
from trulens_eval.schema import FeedbackResult
tru = Tru()
tru.reset_database()

## Get data

In [ ]:
!curl https://www.ipcc.ch/report/ar6/wg2/downloads/report/IPCC_AR6_WGII_Chapter03.pdf --output IPCC_AR6_WGII_Chapter03.pdf

## Create index

In [ ]:
from llama_index.core import (
    VectorStoreIndex,
    StorageContext,
    SimpleDirectoryReader,
)
from llama_index.legacy import ServiceContext
from llama_index.llms.openai import OpenAI

# load documents
documents = SimpleDirectoryReader(
    input_files=["IPCC_AR6_WGII_Chapter03.pdf"]
).load_data()

# initialize service context (set chunk size)
# -- here, we set a smaller chunk size, to allow for more effective re-ranking
llm = OpenAI(model="gpt-3.5-turbo")
service_context = ServiceContext.from_defaults(chunk_size=256, llm=llm)
nodes = service_context.node_parser.get_nodes_from_documents(documents)

# initialize storage context (by default it's in-memory)
storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

In [ ]:
index = VectorStoreIndex(
    nodes, storage_context=storage_context, service_context=service_context
)

## Set up retrievers

In [ ]:
from llama_index.retrievers import BM25Retriever

# retireve the top 10 most similar nodes using embeddings
vector_retriever = index.as_retriever(similarity_top_k=10)

# retireve the top 10 most similar nodes using bm25
bm25_retriever = BM25Retriever.from_defaults(nodes=nodes, similarity_top_k=10)

## Create Hybrid (Custom) Retriever

In [ ]:
from llama_index.retrievers import BaseRetriever

class HybridRetriever(BaseRetriever):
    def __init__(self, vector_retriever, bm25_retriever):
        self.vector_retriever = vector_retriever
        self.bm25_retriever = bm25_retriever
        super().__init__()

    def _retrieve(self, query, **kwargs):
        bm25_nodes = self.bm25_retriever.retrieve(query, **kwargs)
        vector_nodes = self.vector_retriever.retrieve(query, **kwargs)

        # combine the two lists of nodes
        all_nodes = []
        node_ids = set()
        for n in bm25_nodes + vector_nodes:
            if n.node.node_id not in node_ids:
                all_nodes.append(n)
                node_ids.add(n.node.node_id)
        return all_nodes

index.as_retriever(similarity_top_k=5)

hybrid_retriever = HybridRetriever(vector_retriever, bm25_retriever)

## Set up reranker

In [ ]:
from llama_index.postprocessor import SentenceTransformerRerank

reranker = SentenceTransformerRerank(top_n=4, model="BAAI/bge-reranker-base")

In [ ]:
from llama_index.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(
    retriever=hybrid_retriever,
    node_postprocessors=[reranker],
    service_context=service_context,
)

In [ ]:
tru.start_dashboard()

## Initialize Context Relevance checks

Include relevance checks for bm25, vector retrievers, hybrid retriever and the filtered hybrid retriever (after rerank and filter).

This requires knowing the feedback selector for each. You can find this path by logging a run of your application and examining the application traces on the Evaluations page.

Read more in our docs: https://www.trulens.org/trulens_eval/selecting_components/

In [ ]:
from trulens_eval.feedback.provider import OpenAI
from trulens_eval.schema import Select
import numpy as np

# Initialize provider class
openai = OpenAI()

bm25_context = Select.RecordCalls._retriever.bm25_retriever.retrieve.rets[:].node.text
vector_context = Select.RecordCalls._retriever.vector_retriever._retrieve.rets[:].node.text
hybrid_context = Select.RecordCalls._retriever.retrieve.rets[:].node.text
hybrid_context_filtered = Select.RecordCalls._node_postprocessors[0]._postprocess_nodes.rets[:].node.text

# Question/statement relevance between question and each context chunk.
f_context_relevance_bm25 = (
    Feedback(openai.qs_relevance, name = "BM25")
    .on_input()
    .on(bm25_context)
    .aggregate(np.mean)
    )

f_context_relevance_vector = (
    Feedback(openai.qs_relevance, name = "Vector")
    .on_input()
    .on(vector_context)
    .aggregate(np.mean)
    )

f_context_relevance_hybrid = (
    Feedback(openai.qs_relevance, name = "Hybrid")
    .on_input()
    .on(hybrid_context)
    .aggregate(np.mean)
    )

f_context_relevance_hybrid_filtered = (
    Feedback(openai.qs_relevance, name = "Hybrid Filtered")
    .on_input()
    .on(hybrid_context_filtered)
    .aggregate(np.mean)
    )

## Add feedbacks

In [ ]:
tru_recorder = TruLlama(query_engine,
    app_id='Hybrid Retriever Query Engine',
    feedbacks=[f_context_relevance_bm25, f_context_relevance_vector, f_context_relevance_hybrid, f_context_relevance_hybrid_filtered])

In [ ]:
with tru_recorder as recording:
    response = query_engine.query(
    "What is the impact of climate change on the ocean?"
)

## Explore in a Dashboard

In [ ]:
tru.run_dashboard() # open a local streamlit app to explore

# tru.stop_dashboard() # stop if needed

Alternatively, you can run `trulens-eval` from a command line in the same folder to start the dashboard.